In [1]:
from datasets import load_dataset
from transformers import AutoModel
import torch


In [4]:


data = load_dataset("cvejoski/mjp", download_mode="force_redownload", trust_remote_code=True)
data.set_format("torch")
fimmjp = AutoModel.from_pretrained("cvejoski/FIMMJP", trust_remote_code=True, download_mode="force_redownload")

device = "cpu"
if torch.cuda.is_available():
   device = "cuda:0"
elif  torch.backends.mps.is_available():
   device = "mps"

fimmjp = fimmjp.to(device)
fimmjp.eval()




mjp.py:   0%|          | 0.00/7.01k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


6_st_DFR_V=0.zip:   0%|          | 0.00/263k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

FIMMJP(
  (pos_encodings): SineTimeEncoding(
    (linear_embedding): Linear(in_features=1, out_features=1, bias=True)
    (periodic_embedding): Sequential(
      (0): Linear(in_features=1, out_features=57, bias=True)
      (1): SinActivation()
    )
  )
  (ts_encoder): TransformerEncoder(
    (layers): MaskedSequential(
      (0): TransformerBlock(
        (attention_head): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ff): MLP(
          (layers): Sequential(
            (linear_0): Linear(in_features=64, out_features=256, bias=True)
            (activation_0): ReLU()
            (linear_1): Linear(in_features=256, out_features=64, bias=True)
            (activation_1): ReLU()
            (output): Linear(in_features=64, out_features=64, bias=True)
         

In [6]:
batch = {k:v.to(device) for k, v in data["train"][:1].items()}

In [ ]:
with torch.no_grad():
    output = fimmjp(batch)

In [ ]:
output